In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# loading full dataframe
final_df = pd.read_csv('final_df.csv')

In [7]:
# dropping the track_id
final_df.drop(['track_id'], axis=1, inplace=True)
final_df

,title,artist,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Crazy - Single Version,"Patsy Cline, The Jordanaires",country,0.471,0.021,10,-24.385,1,0.0366,0.931000,0.000001,0.1030,0.356,107.796
1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",country,0.754,0.540,5,-8.326,1,0.0266,0.687000,0.000075,0.0868,0.790,101.738
2,I Will Always Love You,Dolly Parton,country,0.446,0.142,9,-13.498,1,0.0335,0.960000,0.001140,0.1010,0.156,135.724
3,Springsteen,Eric Church,country,0.692,0.809,2,-5.935,1,0.0255,0.100000,0.000098,0.1850,0.927,104.023
4,He Stopped Loving Her Today,George Jones,country,0.430,0.173,8,-18.597,1,0.0321,0.594000,0.000111,0.2240,0.365,72.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7910,Alpha Centauri,Fergie,techno,0.569,0.983,1,-4.547,1,0.0816,0.000204,0.907000,0.0547,0.356,129.976
7911,The Future,"CamelPhat, Rebūke",techno,0.599,0.910,6,-7.403,0,0.1070,0.000116,0.876000,0.1500,0.110,128.020
7912,Vibrant,LEVT,techno,0.612,0.905,1,-7.154,0,0.0615,0.000170,0.887000,0.0632,0.398,132.000
7913,Zero Gravity,Ramon Tapia,techno,0.637,0.931,2,-3.093,1,0.0449,0.000198,0.877000,0.3580,0.530,133.993


In [18]:
genre_count = final_df['genre'].value_counts()

### Handling Data Imbalance

In [9]:
# setting a minimum number of sample for each genre
min_genre_count = 800

In [62]:
# genres with samples more than min_genre_count
genre_count[genre_count > min_genre_count].index

Index(['house', 'pop', 'rock'], dtype='object')

In [63]:
# retaining data for genres with samples more than min_genre_count
genre_house = final_df[final_df['genre'] == 'house']
genre_pop = final_df[final_df['genre'] == 'pop']
genre_rock = final_df[final_df['genre'] == 'rock']

In [64]:
# genres with samples less than min_genre_count
genre_count[genre_count < min_genre_count].index

Index(['country', 'metal', 'hiphop', 'techno', 'dance', 'jazz', 'kpop',
       'classical', 'rnb'],
      dtype='object')

In [65]:
# upsampling data for genres with samples less than min_genre_count
genre_country = final_df[final_df['genre']=='country'].sample(min_genre_count, replace=True)
genre_metal = final_df[final_df['genre']=='metal'].sample(min_genre_count, replace=True)
genre_hiphop = final_df[final_df['genre']=='hiphop'].sample(min_genre_count, replace=True)
genre_techno = final_df[final_df['genre']=='techno'].sample(min_genre_count, replace=True)
genre_dance = final_df[final_df['genre']=='dance'].sample(min_genre_count, replace=True)
genre_jazz = final_df[final_df['genre']=='jazz'].sample(min_genre_count, replace=True)
genre_kpop = final_df[final_df['genre']=='kpop'].sample(min_genre_count, replace=True)
genre_classical = final_df[final_df['genre']=='classical'].sample(min_genre_count, replace=True)
genre_rnb = final_df[final_df['genre']=='rnb'].sample(min_genre_count, replace=True)

In [84]:
# concatetnating all genre dataframes after upsampling
upsample_df = pd.concat([genre_house, genre_pop, genre_rock, genre_country, genre_metal, genre_hiphop, genre_techno, genre_dance, genre_jazz,  genre_kpop, genre_classical, genre_rnb], axis=0)

In [86]:
# computing supposed total numbers of tracks after upsampling
genre_count[genre_count > min_genre_count].values.sum()+len(genre_count[genre_count < min_genre_count].index)*min_genre_count

10536

In [87]:
# checking total numbers of tracks in new dataframe
len(upsample_df)

10536